## **Bigcontest 2020**
### NS Shop+ 판매 실적 예측

In [1]:
import numpy as np
import pandas as pd
import os
print("numpy version : ", np.__version__)
print("pandas version : ", pd.__version__)
print(os.listdir())

numpy version :  1.17.2
pandas version :  1.0.5
['2020빅콘테스트 문제데이터(데이터분석분야-챔피언리그).zip', 'EDA.ipynb', '.ipynb_checkpoints', '02_평가데이터', '01_제공데이터']


In [2]:
data = os.listdir('01_제공데이터')
print(data)

['2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx', '2020 빅콘테스트 데이터분석분야-챔피언리그_시청률 데이터.xlsx']


예측 상품 중 판매가 0인 프로그램 실적은 제외, 과거 실적이 없는 경우는 유사 카테고리 혹은 동일 머더코드로 예측.

In [3]:
res_19 = pd.read_excel("01_제공데이터/"+data[0], sheet_name = "rawdata_2019(완)", skiprows=1)
res_19.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


In [4]:
res_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38309 entries, 0 to 38308
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   방송일시    38309 non-null  datetime64[ns]
 1   노출(분)   21525 non-null  float64       
 2   마더코드    38309 non-null  int64         
 3   상품코드    38309 non-null  int64         
 4   상품명     38309 non-null  object        
 5   상품군     38309 non-null  object        
 6   판매단가    38309 non-null  int64         
 7   취급액     37372 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.3+ MB


In [5]:
res_19.isna().sum()

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액        937
dtype: int64

In [6]:
res_19[res_19['취급액'].isna()]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
2318,2019-01-25 19:00:00,60.000000,100507,201567,TCOM_처브치아보험,무형,0,NaN
2845,2019-01-31 18:00:00,60.000000,100507,201567,TCOM_처브치아보험,무형,0,NaN
3332,2019-02-05 18:00:00,60.000000,100507,201567,TCOM_처브치아보험,무형,0,NaN
6570,2019-03-09 00:00:00,20.000000,100324,201095,온라인투어 동남아 나트랑/빈펄랜드 (190309),무형,0,NaN
6571,2019-03-09 00:20:00,20.000000,100324,201095,온라인투어 동남아 나트랑/빈펄랜드 (190309),무형,0,NaN
...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


- **판매가 0원(무형, 보험 상품)은 추정 제외**

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
rating = pd.read_excel("01_제공데이터/"+data[1], skiprows=1)
rating.head()

,시간대,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,...,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31,2019-01-01 to 2019-12-31
0,02:00,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.003
1,02:01,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.012,0.003
2,02:02,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.004
3,02:03,0.0,0.0,0.014,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.017,0.0,0.0,0.0,0.0,0.0,0.000,0.004
4,02:04,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.017,0.0,0.0,0.0,0.0,0.0,0.000,0.004


시간대별로 0101~1231(365일)의 분당 시청률

## 평가데이터
> 취급액 예측(취급액 = 판매단가 x 주문량)

- 결국 주문량을 맞추면 되는것!

In [9]:
sub = pd.read_excel("02_평가데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx", skiprows=1)
sub.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN


## Metric

### MAPE
(예측값-실제값)/예측값 -> 예측값에 대한 오차의 비율

오차의 비율로 모델을 비교할 수 있어 오차 평균의 크기가 크게 차이나는 모델을 비교할 수 있다.